Reinforcement Learning for SMS Messaging to Improve Medication Adherence - Roybal

Import necessary packages

In [ ]:
import sys
import time
import dateutil
from azure.cognitiveservices.personalizer import PersonalizerClient
from azure.cognitiveservices.personalizer.models import RankRequest
from msrest.authentication import CognitiveServicesCredentials
import pandas as pd
import numpy as np
import math
import time
from datetime import datetime
from collections import Counter
import string
import pickle
import json
from dateutil import parser

Imports from Local Python Code Files

In [2]:
from fresh_restart.patient_data import import_pt_data,export_pt_data
from fresh_restart.pillsy_parser import import_Pillsy, find_rewards
from fresh_restart.driverReward import get_reward_update,send_rewards
from fresh_restart.redcap_parser import import_redcap, update_pt_data_with_redcap
from fresh_restart.driverRank import run_ranking, write_sms_history
from fresh_restart.control_disconnection import check_control_disconnectedness

Start Program Timer & Set Up MS Azure Personalizer Client

In [3]:
start_time = time.time()
run_time = datetime.now()
testing_flag = input("Testing with another date? y/n: ")
if testing_flag.lower() == "y":
    print("Set testing run time: ")
    run_time_yyyy_mm_dd_input = input("Enter the testing date: YYYY-MM-DD ")
    timestamp = "10:30 AM -0500 " + run_time_yyyy_mm_dd_input
    run_time = dateutil.parser.parse(timestamp)


personalizer_key = input("ENTER PERSONALIZER KEY: \n")

personalizer_endpoint = input("ENTER PERSONALIZER ENDPOINT: \n")

Defining and Instantiating a Personalizer Client:

Personalizer Keys:
* In the Microsoft Azure Dashboard, navigate to our bwh-pharmacoepi-roybal-dev-use2-cog Cognitive Services page.
* Within the Keys and Endpoint section, copy either Key 1 or Key 2 to enter as the Personalizer Key.

Personalizer Endpoint:
* https://bwh-pharmacoepi-roybal-dev-use2-cog.cognitiveservices.azure.com/

In [4]:
client = PersonalizerClient(personalizer_endpoint, CognitiveServicesCredentials(personalizer_key))

ValueError: Subscription key cannot be None

If we've already initiated the trial, we will have:
* Pre-existing patient dataset in need of reward updates
* Pillsy data from yesterday to determine reward
If this is study initiation, this step will just load an empty patient dictionary and null pillsy dataset.

In [ ]:
pt_data = import_pt_data(run_time)
new_pillsy_data = import_Pillsy(run_time)

# Make sure pt_dict and new_pillsy_data aren't empty or null (i.e. for study initiation).
if not pt_data and not new_pillsy_data:
# Extract list of study_id's in the Pillsy Data.
# From Pillsy data, computes the Rewards to send to Personalizer for each patient's Rank calls from yesterday's run.
    find_rewards(new_pillsy_data,pt_data,run_time)
# Using this updated Pillsy joined Patient data, we format the rewards to Personalizer into a dataframe
    rewards_to_send = get_reward_update(pt_data, run_time)
    send_rewards(rewards_to_send, client, run_time)
    # TODO export pt_data as csv for log of reward data
export_pt_data(pt_data, "reward", run_time)

Call Personalizer to rank action features to find the correct text message to send today.

In [ ]:
redcap_data = import_redcap(run_time)
pt_data = update_pt_data_with_redcap(redcap_data, pt_data, run_time)
export_pt_data(pt_data, "rank", run_time)

for index, patient in pt_data.iterrows():
    if patient["censor"] != 1 and patient["censor_date"] > run_time.date():
        run_ranking(patient, client, run_time)
        #need checks if pass by ref or val

write_sms_history(pt_data, run_time)
export_pt_data(pt_data, "final", run_time)

Evaluate control group for connection issues as well

In [ ]:
check_control_disconnectedness(run_time)

To Do List:
  * Executability by RA / User friendliness
    * [x] - Lily done by using Jupyter notebook - Making this more user friendly than a  Command Line
    * [ ] - JOE TODO / help brainstorm - make jupyter notebook pretty and write a how to document for new non CS person to be able to execute this from Jupyter notebook
    * [ ] - JOE TODO / help brainstorm - Making this more user friendly than a Jupyter Notebook - to do by doing a main.py executable bash script
    * [ ] - JOE TODO / help brainstorm - Hooks into Pillsy/RedCap for data retrieval - need check in with constance for pillsy and ellen for redcap
    * [ ] - JOE TODO / help brainstorm - (probably not feasible) Hooks into SMS Platform to automate text sending - need check in with constance
    * [ ] - JOE TODO / help brainstorm - how to handle the definite 1 human entered variable of personalizer key - file to direct to or? - maybe as marco/elad
    * [ ] - Lily/JOE TODO / help - try to break this code in any way possible
    * [ ] - Lily/JOE TODO / help - debug, unit testing
    * [ ] - Lily/JOE TODO / help - run it fully several times based in dropbox together
    * [ ] - Lily/JOE TODO / help - Make a log file that will report high level information from running this like a run summary
        * elements to include:
            * start and end time of the run
            * how many patients were read in from each import statement
            * how many reward calls were successfull made
            * how many rank calls were successfull made
            * any other meta data that will help us debug and ensure this is all working as planned


